# Исследование игровой индустрии в начале XXI века

- Автор: Уляшев Виталий
- Дата: 13.02.2025 - 19.02.2025

### Цели и задачи проекта

Подготовить данные для статьи о развитии индустрии игр в начале XXI века. Для этого следует изучить датасет полученный от заказчика и провести его предобработку. Сделать обзор игровых платформ за период с 2000 по 2013 год включительно. Выделить топ-7 платформ по количеству игр, выпущенных в течении изучаемого периода. Провести категоризацию игр по оценкам пользователей и экспертов.

### Описание данных

Для решения задачи будем использовать датасет `/datasets/new_games.csv`, который содержит следующую информацию:

- `Name` - название игры.
- `Platform` - название платформы.
- `Year of Release` - год выпуска игры.
- `Genre` - жанр игры.
- `NA sales` - продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` - продажи в Европе (в миллионах проданных копий).
- `JP sales` - продажи в Японии (в миллионах проданных копий).
- `Other sales` - продажи в других странах (в миллионах проданных копий).
- `Critic Score` - оценка критиков (от 0 до 100).
- `User Score` - оценка пользователей (от 0 до 10).
- `Rating` - рейтинг организации ESRB (Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

<a id='start'></a>
### Содержимое проекта

1. [Загрузка данных и знакомство с ними.](#load)


2. [Проверка ошибок в данных и их предобработка.](#pre)

    2.1 [Названия, или метки, столбцов датафрейма.](#name)
  
    2.2 [Типы данных.](#type)
  
    2.3 [Наличие пропусков в данных.](#null)
  
    2.4 [Явные и неявные дубликаты в данных.](#dupl)
  
  
3. [Фильтрация данных.](#filter)


4. [Категоризация данных.](#category)


5. [Итоговый вывод.](#result)

---

<a id='load'></a>
## 1. Загрузка данных и знакомство с ними


In [1]:
# Импортируем библиотеку pandas.

import pandas as pd

In [2]:
# Загрузим данные и присвоим имя датафрейму.

df_games = pd.read_csv('/datasets/new_games.csv')

In [3]:
# Выведем первые 5 строк датафрейма.

df_games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Выведем общую информацию о датафрейме.

df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Датафрейм содержит 16956 строк и 11 столбцов. В столбцах встречаются следующие типы данных:

- **Числовые значения с плавающей запятой (float64)** (4 столбца):
  - `Critic Score`, `Other sales`, `NA sales` показывают оценки критиков и продажи (млн. копий), здесь могут быть значения с дробной частью, поэтому выбранный тип данных представлен корректно.
  - `Year of Release` показывает год выпуска игры, поэтому лучше использовать для этого столбца тип `datetime64`.
- **Строковые данные (object)** (7 столбцов):
  - `Name` показывает название игры, поэтому тип данных подходит.
  - `Platform`, `Genre`, `Rating` также хранят текстовые данные, но так как набор значений в этих столбцах ограничен можно использовать тип `category`.
  - `EU sales`, `JP sales`, `User Score` показывают продажи (млн. копий) и оценки пользователей, для таких данных лучше использовать тип `float64`, как в аналогичных столбцах рассмотренных выше.
  
Названия столбцов не соответствуют стилю snake case.
  
Столбцы `Platform`, `NA sales`, `EU sales`, `JP sales`, `Other sales` не содержат пропусков. Есть незначительное количество пропусков в столбцах `Name` и `Genre`. В столбце `Year of Release` пропусков немного больше, но на первый взгляд, тоже незначительное количество. А вот в столбцах `Rating`, `Critic Score`, `User Score` довольно большое количество пропусков.

После анализа типов данных видно, что столбец `Year of Release` лучше преобразовать в тип `datetime64`. Столбцы `EU sales`, `JP sales`, `User Score` преобразовать в тип `float64`. А в столбцах `Platform`, `Genre`, `Rating` можно использовать тип `category`.

[содержание](#start)

---
<a id='pre'></a>
## 2.  Проверка ошибок в данных и их предобработка

<a id='name'></a>
### 2.1. Названия, или метки, столбцов датафрейма

In [5]:
# Выводим название всех столбцов.

df_games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
# Заменим названия столбцов, чтобы привести их к стилю snake case.

df_games.columns = df_games.columns.str.lower().str.replace(' ', '_')
df_games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

[содержание](#start)

<a id='type'></a>
### 2.2. Типы данных


Разные типы данных имеют столбцы с продажами, хотя должны иметь один тип. Также это касается столбцов с оценками критиков и пользователей. Так могло получиться потому, что данные собирали из разных источников или из-за технических ошибок при составлении датасета.

Так же странно, что у столбца с годом выпуска игры тип данных `float64`. Возможно изначально там был целочислительный тип данных, но из-за наличия пропусков, числа в столбце стали вещественными.

In [7]:
# Преобразуем к типу `datetime64` столбец с годом выпуска игры.

df_games['year_of_release'] = pd.to_datetime(df_games['year_of_release'], format='%Y', errors='coerce')
df_games['year_of_release']

0       2006-01-01
1       1985-01-01
2       2008-01-01
3       2009-01-01
4       1996-01-01
           ...    
16951   2016-01-01
16952   2006-01-01
16953   2016-01-01
16954   2003-01-01
16955   2016-01-01
Name: year_of_release, Length: 16956, dtype: datetime64[ns]

In [8]:
# изучим уникальные значения в столбце 'platform'.

df_games['platform'].unique()

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

In [9]:
# Приведем столбец с названиями платформ к типу `category`.

df_games['platform'] = df_games['platform'].astype('category')

In [10]:
# изучим уникальные значения в столбце 'genre'.

df_games['genre'].unique()

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', nan, 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION',
       'SHOOTER', 'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE',
       'SIMULATION', 'PUZZLE', 'STRATEGY'], dtype=object)

In [11]:
# Приведем столбец с жанром игры к типу `category`.
# Чтобы жанры не повторялись сделаем все значения в этом столбце с нижним регистром.

df_games['genre'] = df_games['genre'].str.lower()
df_games['genre'] = df_games['genre'].astype('category')
df_games['genre'].value_counts()

action          3418
sports          2375
misc            1772
role-playing    1516
shooter         1346
adventure       1323
racing          1273
platform         904
simulation       884
fighting         862
strategy         691
puzzle           590
Name: genre, dtype: int64

In [12]:
# изучим уникальные значения в столбце 'eu_sales'.

df_games['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

In [13]:
# Приведем столбец с продажами в Европе к типу `float64`, заменив текстовые значения на пропуски.

df_games['eu_sales'] = pd.to_numeric(df_games['eu_sales'], errors='coerce')

In [14]:
# изучим уникальные значения в столбце 'jp_sales'.

df_games['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [15]:
# Приведем столбец с продажами в Японии к типу `float64`, заменив текстовые значения на пропуски.

df_games['jp_sales'] = pd.to_numeric(df_games['jp_sales'], errors='coerce')

In [16]:
# изучим уникальные значения в столбце 'user_score'.

df_games['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In [17]:
# Приведем столбец с оценками от пользователей к типу `float64`, заменив текстовые значения на пропуски.

df_games['user_score'] = pd.to_numeric(df_games['user_score'], errors='coerce')

In [18]:
# Проверим полученный результат и примерно оценим количество добавленных пропусков.

df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   name             16954 non-null  object        
 1   platform         16956 non-null  category      
 2   year_of_release  16681 non-null  datetime64[ns]
 3   genre            16954 non-null  category      
 4   na_sales         16956 non-null  float64       
 5   eu_sales         16950 non-null  float64       
 6   jp_sales         16952 non-null  float64       
 7   other_sales      16956 non-null  float64       
 8   critic_score     8242 non-null   float64       
 9   user_score       7688 non-null   float64       
 10  rating           10085 non-null  object        
dtypes: category(2), datetime64[ns](1), float64(6), object(2)
memory usage: 1.2+ MB


Поменяли тип данных в столбце `year_of_release` на `datetime64`, при этом дополнительные пропуски не добавились. 

Поменяли тип данных в столбцах `platform`, `genre` на `category`, дополнительные пропуски не добавились. Избавились от повторяющихся значений, которые относились к одной и той же платформе. Позже, после замены пропусков в столбце `rating`, его тоже можно будет привести к типу `category`.

Поменяли тип данных в столбцах `eu_sales`, `jp_sales`, `user_score` на `float64` с заменой строковых значений на пропуски. В итоге в столбцах `eu_sales` и `jp_sales` добавилось незначительное количество пропусков (6 и 4 соответственно), а в столбце `user_score` добавилось примерно 2500 пропусков. В столбцах `eu_sales` и `jp_sales` были заменены на пропуски значения `unknown`. В столбце `user_score` было заменено значение `tbd`, что означает "будет уточнен позднее". Когда данные появятся в источнике можно будет добавить их в датасет и пересчитать итоговые показатели. Пока будем считать это значение пропуском.

Конкретные значения пропусков будут расчитаны далее.

[содержание](#start)

<a id='null'></a>
### 2.3. Наличие пропусков в данных



In [19]:
# Считаем количество пропусков в абсолютных значениях.

df_games.isna().sum().sort_values()

platform              0
na_sales              0
other_sales           0
name                  2
genre                 2
jp_sales              4
eu_sales              6
year_of_release     275
rating             6871
critic_score       8714
user_score         9268
dtype: int64

In [20]:
# Считаем количество пропусков в относительных значениях.

df_games.isna().mean().sort_values()

platform           0.000000
na_sales           0.000000
other_sales        0.000000
name               0.000118
genre              0.000118
jp_sales           0.000236
eu_sales           0.000354
year_of_release    0.016218
rating             0.405225
critic_score       0.513918
user_score         0.546591
dtype: float64

In [21]:
# Посмотрим на строки с пропущенными значениями в столбце `name`.

df_games[df_games['name'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
661,NaN,GEN,1993-01-01,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14439,NaN,GEN,1993-01-01,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Сразу видим, что сюда попали пропущенные значения из столбца с жанром игры, в этом столбце такое же пропущенное количество значений, 2 пропуска. Эти строки относятся к платформе "GEN", всего игр на этой платформе 29. Похоже, что эти пропуски случайны и вызваны технической ошибкой. Так как эти строки не попадают в нашу выборку из-за раннего года выпуска, то ничего с ними делать не будем или можно вообще их удалить.

In [22]:
# Посмотрим на строки с пропущенными значениями в столбце `year_of_release`.

df_games[df_games['year_of_release'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
183,Madden NFL 2004,PS2,NaT,sports,4.26,0.26,0.01,0.71,94.0,8.5,E
379,FIFA Soccer 2004,PS2,NaT,sports,0.59,2.36,0.04,0.51,84.0,6.4,E
458,LEGO Batman: The Videogame,Wii,NaT,action,1.80,0.97,0.00,0.29,74.0,7.9,E10+
477,wwe Smackdown vs. Raw 2006,PS2,NaT,fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN
611,Space Invaders,2600,NaT,shooter,2.36,0.14,0.00,0.03,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16609,PDC World Championship Darts 2008,PSP,NaT,sports,0.01,0.00,0.00,0.00,43.0,NaN,E10+
16641,Freaky Flyers,GC,NaT,racing,0.01,0.00,0.00,0.00,69.0,6.5,T
16685,Inversion,PC,NaT,shooter,0.01,0.00,0.00,0.00,59.0,6.7,M
16695,Hakuouki: Shinsengumi Kitan,PS3,NaT,adventure,0.01,0.00,0.00,0.00,NaN,NaN,NaN


В этом столбце 275 пропусков, это 1.6% от всех значений. Предположить из-за чего возникли эти пропуски трудно, так как не видно явной взаимосвязи с другими столбцами и пропусками. Похоже что эти пропуски случайны и вызваны технической ошибкой. Количество пропусков относительно всех строк небольшое, можно их оставить или удалить. Но из названий игр и платформ видно, что значительное количество из этих строк относятся к нашей выборке. Если в названии игры указан год, то его можно восстановить. Можно и с остальными играми разобраться с помощью открытых источников, осуществляя поиск вручную, основываясь на названии игры и платформы, но такой метод может занять много времени.

In [23]:
# Посмотрим на строки с пропущенными значениями в столбце `eu_sales`.

df_games[df_games['eu_sales'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
446,Rhythm Heaven,DS,2008-01-01,misc,0.55,NaN,1.93,0.13,83.0,9.0,E
802,Dead Rising,X360,2006-01-01,action,1.16,NaN,0.08,0.20,85.0,7.6,M
1131,Prince of Persia: Warrior Within,PS2,2004-01-01,action,0.54,NaN,0.00,0.22,83.0,8.5,M
1132,Far Cry 4,XOne,2014-01-01,shooter,0.80,NaN,0.01,0.14,82.0,7.5,M
1394,Sonic Advance 3,GBA,2004-01-01,platform,0.74,NaN,0.08,0.06,79.0,8.4,E
1612,Ratatouille,DS,2007-01-01,action,0.49,NaN,0.00,0.14,NaN,NaN,NaN


Здесь 6 пропусков и все они относятся к нашей выборке. Возникновение пропусков скорее всего случайно и вызвано технической ошибкой. Можно восстановить их используя среднее значение. Такой подход не исказит итоговые вычисления, потому что пропусков очень мало.

In [24]:
# Посмотрим на строки с пропущенными значениями в столбце `jp_sales`.

df_games[df_games['jp_sales'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
467,Saints Row 2,X360,2008-01-01,action,1.94,0.79,NaN,0.28,81.0,8.1,M
819,UFC 2009 Undisputed,X360,2009-01-01,fighting,1.48,0.39,NaN,0.19,83.0,7.9,T
1379,Hello Kitty Party,DS,2007-01-01,misc,0.78,0.51,NaN,0.12,NaN,NaN,E
4732,Castlevania: The Dracula X Chronicles,PSP,2007-01-01,platform,0.22,0.09,NaN,0.07,80.0,7.8,T


Здесь 4 пропуска и все они относятся к нашей выборке. Возникновение пропусков скорее всего случайно и вызвано технической ошибкой. Можно восстановить их используя среднее значение. Такой подход не исказит итоговые вычисления, потому что пропусков очень мало.

In [25]:
# Посмотрим на строки с пропущенными значениями в столбце `critic_score`.

df_games[df_games['critic_score'].isna()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1,Super Mario Bros.,NES,1985-01-01,platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
4,Pokemon Red/Pokemon Blue,GB,1996-01-01,role-playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989-01-01,puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
9,Duck Hunt,NES,1984-01-01,shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN
10,Nintendogs,DS,2005-01-01,simulation,9.05,10.95,1.93,2.74,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016-01-01,action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006-01-01,sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016-01-01,adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003-01-01,platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


В столбце с оценками от критиков 8714 пропусков, что составляет 51% от всех строк. При детальном рассмотрении таблицы можно заметить, что игры ранних годов выпуска не имеют оценок. Можно предположить, что в то время еще не собирались такие данные, поэтому они отсутствуют. В играх за последний год (в таблице это 2016), так же отсутствуют оценки критиков. В этом случае можно предположить, что они еще не сформированны. 

Если посмотреть в разрезе нашей выборки, то там по прежнему будет много пропусков, почти половина от всех значений. Значительное большинстство этих пропусков относятся к играм, у которых низкие показатели продаж (меньше 0.1 млн копий). Не похоже, что это случайные пропуски. Возможно такие игры были проигнорированы или просто не замечены критиками. Есть и единичные случаи у которых продажи меньше 0.1 млн копий, но при этом с высокой оценкой. Можно предположить, что если продажи меньше 0.1 млн копий и отсутствует оценки критиков, то эти оценки низкие.

Можно встретить пропуски у игр и с большими продажами, в таком случае причина ошибки скорее всего техническая.

Из-за того что в этом столбце очень много пропущенных значений, оставим его без изменений, чтобы не искажать результаты вычислений.

В столбцах `user_score` и `rating` похожие наблюдения и выводы как в столбце `critic_score`. Столбец `user_score` содержит 9268 пропусков, что составляет 55% от всех значений. Столбец `rating` содержит 6871 пропусков, что составляет 41% от всех значений. Практически во всех случаях пропуски в столбцах `user_score`, `rating` и `critic_score` находятся на одних и тех же строках. Есть строки где оценка критиков присутствует, а оценки от пользователей нет. Это еще одна причина не удалять пропущенные строки, чтобы не терять данные.

Поэтому столбец `user_score` тоже оставим без изменений, чтобы не искажать результаты. Столбец `rating` также можно оставить без изменений, так как он вообще не понадобиться нам для вычислений, или заменить пропущенные строки индикатором.

In [26]:
# Восстановим некоторые даты из названия игр. Обычно год стоит в конце названия игры,
# поэтому создадим новый столбец и вынесем туда последние 4 символа из названия игры.
# Потом преобразуем этот столбец в числовой тип данных и избавимся от значений где нет цифр.

df_games['year_from_game'] = df_games['name'].str.slice(-4)
df_games['year_from_game'] = pd.to_numeric(df_games['year_from_game'], errors='coerce')

# Функция возвращает значение из ранее созданного столбца, в столбец с годом релиза,если там есть пропуск.
# Так как в этот столбец попали значения из игр про будущее с 
# слишком далеким годом и небольшие значения, говорящие, что это 2 или 3 часть игры,
# поэтому ограничемся диапозоном равным нашей выборке.

display(df_games[(df_games['year_from_game'] >= 2000) & (df_games['year_from_game'] <= 2013) & (df_games['year_of_release'].isna())])

def extract_year(row):
    
    if pd.isna(row['year_of_release']):
        if row['year_from_game'] >= 2000 and row['year_from_game'] <= 2013:
            return pd.to_datetime(row['year_from_game'], format='%Y')
        else:
            return row['year_of_release'] 
    else:
        return row['year_of_release']
    
df_games['year_of_release'] = df_games.apply(extract_year, axis=1)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,year_from_game
183,Madden NFL 2004,PS2,NaT,sports,4.26,0.26,0.01,0.71,94.0,8.5,E,2004.0
379,FIFA Soccer 2004,PS2,NaT,sports,0.59,2.36,0.04,0.51,84.0,6.4,E,2004.0
477,wwe Smackdown vs. Raw 2006,PS2,NaT,fighting,1.57,1.02,0.00,0.41,NaN,NaN,NaN,2006.0
1664,NASCAR Thunder 2003,PS2,NaT,racing,0.60,0.46,0.00,0.16,84.0,8.7,E,2003.0
3524,Madden NFL 2002,XB,NaT,sports,0.53,0.02,0.00,0.03,90.0,8.1,E,2002.0
4830,NFL GameDay 2003,PS2,NaT,sports,0.20,0.15,0.00,0.05,60.0,NaN,E,2003.0
5219,NBA Live 2003,XB,NaT,sports,0.31,0.04,0.00,0.01,82.0,8.8,E,2003.0
5727,All-Star Baseball 2005,PS2,NaT,sports,0.16,0.12,0.00,0.04,72.0,8.6,E,2005.0
5966,NBA Live 2003,GC,NaT,sports,0.23,0.06,0.00,0.01,82.0,8.2,E,2003.0
9042,All-Star Baseball 2005,XB,NaT,sports,0.11,0.03,0.00,0.01,75.0,8.8,E,2005.0


In [27]:
# Удалим созданный столбец.

df_games = df_games.drop('year_from_game', axis=1)

In [28]:
# Удалим строки с пропусками в столбце 'name', мы все равно не знаем,
# что это за игры и они не относятся к нашей выборке.

df_games_new = df_games.copy()
df_games_new = df_games_new.dropna(subset=['name'])

In [29]:
# Удаляем строки которые имеют пропуски в столбце 'year_of_release'.
# И приведем этот столбец к типу данных 'int'.

df_games_new = df_games_new.dropna(subset=['year_of_release'])
df_games_new['year_of_release'] = df_games_new['year_of_release'].dt.year

In [30]:
# С помощью функции заменим пропуски в столбце 'eu_sales'
#  на среднее значение в зависимости от названия платформы и года выхода игры.

def fill_eu_sales(row):
    if pd.isna(row['eu_sales']):
        group = df_games_new[(df_games_new['platform'] == row['platform']) &
                        (df_games_new['year_of_release'] == row['year_of_release'])]
        return round(group['eu_sales'].mean(), 2)
    else:
        return row['eu_sales']
    
df_games_new['eu_sales'] = df_games_new.apply(fill_eu_sales, axis=1)

In [31]:
# С помощью функции заменим пропуски в столбце 'jp_sales',
# на среднее значение в зависимости от названия платформы и года выхода игры.

def fill_jp_sales(row):
    if pd.isna(row['jp_sales']):
        group = df_games_new[(df_games_new['platform'] == row['platform']) &
                        (df_games_new['year_of_release'] == row['year_of_release'])]
        return round(group['jp_sales'].mean(), 2)
    else:
        return row['jp_sales']
    
df_games_new['jp_sales'] = df_games_new.apply(fill_jp_sales, axis=1)

In [32]:
# Заполним пустые значения в столбце 'rating' индикатором 'unknown'.
# После переведем этот столбец к типу данных category.

df_games_new['rating'] = df_games_new['rating'].fillna('unknown')
df_games_new['rating'] = df_games_new['rating'].astype('category')

In [33]:
# Заменим пустые значения в столбце 'critic_score' индикатором "-1".

df_games_new['critic_score'] = df_games_new['critic_score'].fillna(-1)

In [34]:
# Заменим пустые значения в столбце 'user_score' индикатором "-1".

df_games_new['user_score'] = df_games_new['user_score'].fillna(-1)

In [35]:
# Проверим информацию после преобразований.

df_games_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16694 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16694 non-null  object  
 1   platform         16694 non-null  category
 2   year_of_release  16694 non-null  int64   
 3   genre            16694 non-null  category
 4   na_sales         16694 non-null  float64 
 5   eu_sales         16694 non-null  float64 
 6   jp_sales         16694 non-null  float64 
 7   other_sales      16694 non-null  float64 
 8   critic_score     16694 non-null  float64 
 9   user_score       16694 non-null  float64 
 10  rating           16694 non-null  category
dtypes: category(3), float64(6), int64(1), object(1)
memory usage: 1.2+ MB


Восстановили некоторые строки в столбце `year_of_release` извлекая год из названия игры. Оставшиеся 260 пропусков удалили и привели к типу данных `int64`.

Удалили 2 строчки без названия игры.

Заменили пропуски в столбцах `eu_sales` и `jp_sales` на среднее значение в зависимости от названия платформы и года выхода игры.

Заменили пропуски в столбце `rating` на значение индикатор `unknown` и после перевели этот столбец к типу данных `category`.

Заменили пропуски в столбцах `critic_score` и `user_score` на идикатор "-1".

[содержание](#start)

<a id='dupl'></a>
### 2.4. Явные и неявные дубликаты в данных


Частично мы уже провели работу по нормализации с некоторыми столбцами, например со столбцом `genre`. Рассмотрим подробнее другие столбцы.

In [36]:
# Рассмотрим уникальные значения в столбце 'platform'.
# 'unique()' показывает только начало и конец списка, поэтому применяю метод 'value_counts()'

df_games_new['platform'].value_counts()

PS2     2161
DS      2148
PS3     1330
Wii     1305
X360    1251
PSP     1213
PS      1208
PC       972
GBA      826
XB       821
GC       550
3DS      522
PSV      434
PS4      395
N64      320
XOne     251
SNES     241
SAT      174
WiiU     147
2600     118
NES      100
GB        97
DC        52
GEN       27
NG        12
SCD        6
WS         6
3DO        3
TG16       2
PCFX       1
GG         1
Name: platform, dtype: int64

Может показаться, что здесь есть неявные дубликаты, например платформы 'GG', 'PCFX', '3DO' из-за малого количества игр и схожим названием с другими платформами. Но если посмотреть названия игр выпущенных на этих платформах и посмотреть информацию в интернет-источниках, то станет ясно, что это не дубликаты.

In [37]:
# Рассмотрим уникальные значения в столбце 'rating'.

df_games_new['rating'].value_counts()

unknown    6783
E          3977
T          2948
M          1558
E10+       1415
EC            8
K-A           3
AO            1
RP            1
Name: rating, dtype: int64

In [38]:
# Вопрос возникает только по рейтингу 'K-A'. Данные из интернет-источников говорят,
# что это бывшее название рейтинга 'E', поэтому заменим значение 'K-A' на 'E'.

df_games_new['rating'] = df_games_new['rating'].str.replace('K-A', 'E')
df_games_new['rating'] = df_games_new['rating'].astype('category')

In [39]:
# Остался столбец 'name', его трудно просмотреть визуально из-за большого количества игр.
# Приведем все названия к нижнему регистру.

df_games_new['name'] = df_games_new['name'].str.lower()

# Удалим пробелы в начале и в конце строк.

df_games_new['name'] = df_games_new['name'].str.strip()

In [40]:
# Отсортируем датафрейм по всем столбцам и выведем явные дубликаты.

df_games_sorted = df_games_new.sort_values(by=df_games_new.columns.tolist())
df_games_sorted[df_games_sorted.duplicated()]

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
15302,11eyes: crossover,X360,2009,adventure,0.00,0.00,0.02,0.00,-1.0,-1.0,unknown
4861,18 wheeler: american pro trucker,PS2,2001,racing,0.20,0.15,0.00,0.05,61.0,5.7,E
13099,4 elements,PC,2009,puzzle,0.00,0.04,0.00,0.01,-1.0,7.4,E
5236,"999: nine hours, nine persons, nine doors",DS,2009,adventure,0.31,0.00,0.03,0.02,-1.0,-1.0,unknown
4959,alpha protocol,X360,2010,role-playing,0.23,0.12,0.00,0.04,63.0,7.2,M
...,...,...,...,...,...,...,...,...,...,...,...
16104,yoake yori ruriiro na portable,PSP,2010,adventure,0.00,0.00,0.02,0.00,-1.0,-1.0,unknown
10662,yu-gi-oh! 5d's wheelie breakers,Wii,2009,racing,0.09,0.01,0.00,0.01,-1.0,-1.0,unknown
2909,yu-gi-oh! the falsebound kingdom,GC,2002,strategy,0.49,0.13,0.07,0.02,-1.0,-1.0,unknown
6696,zoo resort 3d,3DS,2011,simulation,0.11,0.09,0.03,0.02,-1.0,-1.0,E


In [41]:
# Оставим только первые записи повторяющихся строк, остальные удалим.

df_games_no_dubl = df_games_sorted.drop_duplicates()
df_games_no_dubl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16459 entries, 4782 to 9260
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16459 non-null  object  
 1   platform         16459 non-null  category
 2   year_of_release  16459 non-null  int64   
 3   genre            16459 non-null  category
 4   na_sales         16459 non-null  float64 
 5   eu_sales         16459 non-null  float64 
 6   jp_sales         16459 non-null  float64 
 7   other_sales      16459 non-null  float64 
 8   critic_score     16459 non-null  float64 
 9   user_score       16459 non-null  float64 
 10  rating           16459 non-null  category
dtypes: category(3), float64(6), int64(1), object(1)
memory usage: 1.2+ MB


Работа с неявными дупликатами:
- на этапе преобразования типа данных все значения в столбце `genre` были приведены к нижнему регистру, чтобы избавиться от неявных дупликатов.
- в столбце `rating` заменили одно значение, которое являлось неявным дупликатом.
- в столбце `name` все значения привели к нижнему регистру и удалили лишние пробелы.

После было обнаружено и удалено 235 явных дупликатов.

In [42]:
# Сначало сохраним количество строк в изначальном датафрейме и в финальной версии.
# После посчитаем количество удалённых строк в абсолютном и относительном значениях.

initially_row = df_games.shape[0]
final_row = df_games_no_dubl.shape[0]
count_delet_row = initially_row - final_row
share_delet_row = round(count_delet_row / initially_row, 2)
print(f'количество удаленных строк в абсолютном значении: {count_delet_row}')
print(f'количество удаленных строк в относительном значении: {share_delet_row}')

количество удаленных строк в абсолютном значении: 497
количество удаленных строк в относительном значении: 0.03


Названия столбцов привели к стилю snake case. Изменили типы данных:
- в столбце `year_of_release` на `int64`.
- в столбцах `platform`, `genre` и `rating` на `category`.
- в столбцах `eu_sales`, `jp_sales`, `user_score` на `float64`.

Удалили 2 строчки без названия игр и 260 пропусков из столбца `year_of_release`.

Изучили пропуски в датафрейме и выявили большое количество пропусков в столбцах `critic_score`, `user_score`, `rating`. Пропуски в этих столбцах составляют примерно половину от всех строк. Из-за слишком большого количества пропусков не стали их удалять, чтобы не искажать вычисления, заменили эти пропуски индикатором. Также были обнаружены пропуски в столбце `year_of_release` не более 1.6%, небольшую часть удалось восстановить опираясь на название игр, где был указан год игры. Незначительные пропуски были обнаружены в столбцах `eu_sales`, `jp_sales`. Их полностью восстановили средним значением в зависимости от платформы и года выпуска игры.

Провели работу с неявными дупликатами, после чего обнаружили и удалили 235 явных дупликатов. Общее количество удаленных строк - 497, что составляет 3% от изначального количества строк.

[содержание](#start)

---
<a id='filter'></a>
## 3. Фильтрация данных


In [43]:
# Создадим новый датафрейм в который войдут игры периода с 2000 по 2013 год включительно.

df_actual = df_games_no_dubl[(df_games_no_dubl['year_of_release'] >= 2000) &
                            (df_games_no_dubl['year_of_release'] <= 2013)]
df_actual

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
8460,.hack//g.u. vol.1//rebirth,PS2,2006,role-playing,0.00,0.00,0.17,0.00,-1.0,-1.0,unknown
7182,.hack//g.u. vol.2//reminisce,PS2,2006,role-playing,0.11,0.09,0.00,0.03,-1.0,-1.0,unknown
8719,.hack//g.u. vol.2//reminisce (jp sales),PS2,2006,role-playing,0.00,0.00,0.16,0.00,-1.0,-1.0,unknown
8410,.hack//g.u. vol.3//redemption,PS2,2007,role-playing,0.00,0.00,0.17,0.00,-1.0,-1.0,unknown
1575,.hack//infection part 1,PS2,2002,role-playing,0.49,0.38,0.26,0.13,75.0,8.5,T
...,...,...,...,...,...,...,...,...,...,...,...
7233,zumba fitness: world party,Wii,2013,misc,0.11,0.10,0.00,0.02,-1.0,-1.0,E
6970,zumba fitness: world party,XOne,2013,misc,0.17,0.05,0.00,0.02,73.0,6.2,E
15739,zwei!!,PSP,2008,role-playing,0.00,0.00,0.02,0.00,-1.0,-1.0,unknown
13138,zyuden sentai kyoryuger: game de gaburincho!!,3DS,2013,action,0.00,0.00,0.05,0.00,-1.0,-1.0,unknown


In [44]:
group_year = df_actual.groupby('year_of_release')['name'].count()
group_year

year_of_release
2000     350
2001     482
2002     830
2003     779
2004     764
2005     941
2006    1008
2007    1198
2008    1429
2009    1426
2010    1255
2011    1137
2012     653
2013     544
Name: name, dtype: int64

[содержание](#start)

---
<a id='category'></a>
## 4. Категоризация данных

In [45]:
# Применим функцию, которая разделит на категории оценки пользователей.

def category_user(score):
    if 0 <= score < 3:
        return 'низкая оценка'
    elif 3 <= score < 8:
        return 'средняя оценка'
    elif 8 <= score <= 10:
        return 'высокая оценка'
    else:
        return 'без оценки'

df_actual_new = df_actual.copy()    
df_actual_new['category_user_score'] = df_actual_new['user_score'].apply(category_user)

In [46]:
# Применим функцию, которая разделит на категории оценки критиков.

def category_critic(score):
    if 0 <= score < 30:
        return 'низкая оценка'
    elif 30 <= score < 80:
        return 'средняя оценка'
    elif 80 <= score <= 100:
        return 'высокая оценка'
    else:
        return 'без оценки' 
 
df_actual_new['category_critic_score'] = df_actual_new['critic_score'].apply(category_critic)

In [47]:
# Считаем количество игр в каждой категории.

group_user = df_actual_new.groupby('category_user_score')['name'].count()
group_user

category_user_score
без оценки        6304
высокая оценка    2293
низкая оценка      116
средняя оценка    4083
Name: name, dtype: int64

In [48]:
# Считаем количество игр в каждой категории.

group_critic = df_actual_new.groupby('category_critic_score')['name'].count()
group_critic

category_critic_score
без оценки        5616
высокая оценка    1698
низкая оценка       55
средняя оценка    5427
Name: name, dtype: int64

In [49]:
# Считаем количество строк для каждой платформы и выводим первые 7 платформ.

df_actual_new['platform'].value_counts().head(7)

PS2     2134
DS      2121
Wii     1275
PSP     1181
X360    1123
PS3     1087
GBA      811
Name: platform, dtype: int64

[содержание](#start)

---
<a id='result'></a>
## 5. Итоговый вывод

Оригинальный датафрейм содержит 16956 строк и 11 столбцов. В нем изменили типы данных:
- в столбце `year_of_release` на `int64`.
- в столбцах `platform`, `genre` и `rating` на `category`.
- в столбцах `eu_sales`, `jp_sales`, `user_score` на `float64`.

Изучили пропуски в датафрейме и выявили большое количество пропусков в столбцах `critic_score`, `user_score`, `rating`. Пропуски в этих столбцах составляют примерно половину от всех строк. Из-за слишком большого количества пропусков не стали их удалять, чтобы не искажать вычисления, просто заменили индикатором "-1". Также были обнаружены пропуски в столбце `year_of_release` не более 1.6%, небольшую часть удалось восстановить опираясь на название игр, где был указан год игры, остальные пропуски удалили. Незначительные пропуски были обнаружены в столбцах `eu_sales`, `jp_sales`. Их полностью восстановили средним значением в зависимости от платформы и года выпуска игры.

Провели работу с неявными дупликатами, после чего обнаружили и удалили 235 явный дупликат. Ранее было удалено еще 2 строчки в которых отсутствовали названия игр и 260 строк в которых отсутствовал год выпуска игры. Общее количество удаленных строк - 497, что составляет 3% от изначального количества строк. После отфильтровали датафрейм оставив в нем только игры с 2000 по 2013 год включительно, так как для исследования нас интересует именно этот период. Итоговое количество строк в новом датафрейме получилось 12796.

Добавили новые столбцы `category_user_score` и `category_critic_score` в которых провели категоризацию оценок, разделив их на 4 вида: высокие, средние, низкие и без оценки. Так как новые столбцы создавались на основе столбцов `critic_score` и `user_score` с индикатором, заменяющий пропуски, то соответственно в новых столбцах половина всех значений "без оценки". Из полученных результатов можно заметить, что преобладают средние оценки. Следом по количеству идут высокие оценки и еще реже пользователи и критики ставили низкие оценки.

Выделили топ-7 платформ по количеству игр, откуда видно, что лидерами в этой области, с небольшой разницей, являются `PS2` и `DS` выпустившие 2134 и 2121 игр соответственно.

In [50]:
# Итоговая информация о датафрейме.

df_actual_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12796 entries, 8460 to 9260
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   name                   12796 non-null  object  
 1   platform               12796 non-null  category
 2   year_of_release        12796 non-null  int64   
 3   genre                  12796 non-null  category
 4   na_sales               12796 non-null  float64 
 5   eu_sales               12796 non-null  float64 
 6   jp_sales               12796 non-null  float64 
 7   other_sales            12796 non-null  float64 
 8   critic_score           12796 non-null  float64 
 9   user_score             12796 non-null  float64 
 10  rating                 12796 non-null  category
 11  category_user_score    12796 non-null  object  
 12  category_critic_score  12796 non-null  object  
dtypes: category(3), float64(6), int64(1), object(3)
memory usage: 1.1+ MB


[содержание](#start)